In [1]:
import bayes3d as b
import jax.numpy as jnp
import jax
import os
import matplotlib.pyplot as plt
import matplotlib

In [2]:
bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = b.utils.ycb_loader.get_test_img('54', '1', bop_ycb_dir)
# b.RENDERER.add_mesh_from_file(os.path.join(b.utils.get_assets_dir(), "sample_objs/cube.obj"), scaling_factor=1.0/10000000000.0)

In [4]:
intrinsics = b.Intrinsics(
    rgbd.intrinsics.height, rgbd.intrinsics.width,
    rgbd.intrinsics.fx, rgbd.intrinsics.fx,
    rgbd.intrinsics.width/2, rgbd.intrinsics.height/2,
    rgbd.intrinsics.near, rgbd.intrinsics.far
)
print(intrinsics)

Intrinsics(height=480, width=640, fx=1066.778, fy=1066.778, cx=320.0, cy=240.0, near=0.01, far=2.0)


In [3]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [4]:
IDX_1 = 1
IDX_2 = 11
frames = 40
devices = 1
objs = 2
dots = 5000 * objs
pc = jnp.array(b.RENDERER.meshes[IDX_1].vertices)
b.show_cloud("1",pc)

In [ ]:
# set up the scene graph

box_dims = [b.utils.aabb(m.vertices)[0] for m in meshes]
box_dims = jnp.array(box_dims)

absolute_poses = jnp.array([
    jnp.eye(4),
    jnp.eye(4),
    jnp.eye(4),
    jnp.eye(4),
    jnp.eye(4),
])

parents = jnp.array([
    -1, 0, 0, 0, 0
])

contact_params = jnp.array(
    [
        [0.0, 0.0, jnp.pi/4],
        [-1.0, -0.5, jnp.pi/4],
        [-0.2, 0.1, jnp.pi/2],
        [2.0, -1.0, jnp.pi/2],
        [1.0, -4.0, jnp.pi/2],
    ]
)

face_parents = jnp.array([2,2,2,2,2])
face_child = jnp.array([0,0,0,0,0])

poses = b.scene_graph.poses_from_scene_graph_jit(
    absolute_poses, box_dims, parents, contact_params, face_parents, face_child
)

pos = jnp.array([[10.0, -15.0, 5.0]])
target = jnp.array([0.0, 0.0, 0.0])
up = jnp.array([0.0, 0.0, 1.0])

camera_pose = b.t3d.transform_from_pos_target_up(pos, target, up)



poses_in_camera_frame = b.t3d.inverse_pose(camera_pose) @ poses

In [5]:
rot = b.t3d.rotation_from_axis_angle(jnp.array([1,0,0]),0)
translation = jnp.array([0,0,2.3/2])
transform = b.t3d.transform_from_rot_and_pos(rot, translation)
transform

Array([[1.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 1.15],
       [0.  , 0.  , 0.  , 1.  ]], dtype=float32)

In [6]:
pc2 = b.t3d.apply_transform(jnp.array(b.RENDERER.meshes[IDX_2].vertices), transform)
b.show_cloud("2",pc2)

In [7]:
pc = jnp.vstack((pc,pc2))

lifetime = 5 #keep 1-1/5 of the dots after every frame update
point_rad = 5


pc_subsample_start = pc[jax.random.choice(jax.random.PRNGKey(10), jnp.arange(pc.shape[0]), shape=(dots,) )] #want 1000 dots total
pc_replacements = pc[jax.random.choice(jax.random.PRNGKey(0), jnp.arange(pc.shape[0]), shape=(frames,dots//lifetime) )]

pc_subsamples = jnp.zeros((frames,*pc_subsample_start.shape))
pc_subsamples = pc_subsamples.at[0,...].set(pc_subsample_start)
for i in range(1,frames):
    pc_subsamples = pc_subsamples.at[i,...].set(pc_subsamples[i-1,...])
    sampled_indices = jax.random.choice(jax.random.PRNGKey(i), jnp.arange(dots), shape=(dots//lifetime,) )
    pc_subsamples = pc_subsamples.at[i,sampled_indices,...].set(pc_replacements[i,...])

# pc_vec = pc_vec.reshape(2, -1, 3)
# pc_vec.shape
# print(pc_vec.shape)
pc_subsamples = pc_subsamples.reshape(frames,2,-1,3)
print(pc_subsamples.shape)

(40, 2, 5000, 3)


In [8]:
#b.show_cloud("1", pc_subsamples[0])

In [9]:
poses_1 = jnp.array([b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
        jnp.array([0.0, 2.0, 0.0]),
        jnp.array([0.0, 0.0, 0.0]),
        jnp.array([0.0, 0.0, 1.0]),
)) @ b.t3d.transform_from_axis_angle(jnp.array([0.0, 0.0, 1.0]), angle) for angle in jnp.linspace(0.0, 4*jnp.pi, frames)])

# poses_2 = jnp.array([b.t3d.inverse_pose(
#     b.t3d.transform_from_rot_and_pos(rot, translation)
# ) @ b.t3d.transform_from_axis_angle(jnp.array([1.0, 0.0, 1.0]), angle) for angle in jnp.linspace(0.0, 4*jnp.pi, frames)])

poses_2 = jnp.array([b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
        jnp.array([0.0, 2.0, 0.0]),
        jnp.array([0.0, 0.0, 0.0]),
        jnp.array([0.0, 0.0, 1.0]),
)) @ b.t3d.transform_from_axis_angle(jnp.array([0.0, 0.0, 1.0]), angle) for angle in jnp.linspace(0.0, 4*jnp.pi, frames)]) @ transform

poses=jnp.stack((poses_1,poses_2),axis=1)
print(poses.shape)


(40, 2, 4, 4)


In [10]:
def circles(flips_xy, radius):
    centers = jnp.array((flips_xy>0).nonzero(size=5000,fill_value=jnp.inf))
    x,y = jnp.meshgrid(jnp.arange(flips_xy.shape[1]),jnp.arange(flips_xy.shape[0]))
    xymesh = jnp.array([y,x])
    distances_to_keypoints = (
        jnp.linalg.norm(xymesh[:, :,:,None] - centers[:,None, None,:],
        axis=0
    ))
    index_of_nearest_keypoint = distances_to_keypoints.argmin(2)
    distance_to_nearest_keypoints = distances_to_keypoints.min(2)
    DISTANCE_THRESHOLD = radius
    valid_match_mask = (distance_to_nearest_keypoints < DISTANCE_THRESHOLD)[...,None]
    return valid_match_mask

def render_point_light(pose, pc_to_render, key):
    pc_in_camera_frame = []
    for o in range(len(pose)):
        pc_in_camera_frame.append(b.t3d.apply_transform(pc_to_render[o], pose[o]))
    pc_in_camera_frame = jnp.array(pc_in_camera_frame).reshape(-1,3)
    
    img = b.render_point_cloud(pc_in_camera_frame, intrinsics)

    rendered_image = b.RENDERER.render_jax(pose, jnp.array([jnp.int32(IDX_1),jnp.int32(IDX_2)]))[:,:,:3]

    mask = (rendered_image[:,:,2] < intrinsics.far)
    matches = (jnp.abs(img[:,:,2] - rendered_image[:,:,2]) < 0.05)
    flips = (jax.random.uniform(key,shape=matches.shape) < 0.0005)
    
    final_no_noise = circles(mask * matches,point_rad)
    final_with_noise = circles(mask * matches + (1.0 - mask) * flips, point_rad)

    return final_no_noise, final_with_noise

gpus = jax.devices('gpu')
render_point_light_parallel_jit = jax.jit(jax.vmap(render_point_light, in_axes=(0,0, 0)))

In [11]:
key = jax.random.PRNGKey(100)
keys = jax.random.split(jax.random.PRNGKey(100), poses.shape[0])
images_no_noise, images = render_point_light_parallel_jit(poses, pc_subsamples, keys)

In [12]:
viz = [b.get_depth_image(1.0 - point_light_image * 1.0, cmap=matplotlib.colormaps['Greys']) for point_light_image in images ]
b.make_gif_from_pil_images(viz, "out_noise_scene.gif")

In [13]:
viz[0].save('out_frame_scene.png')

In [14]:
viz = [b.get_depth_image(1.0 - point_light_image * 1.0, cmap=matplotlib.colormaps['Greys']) for point_light_image in images_no_noise ]
b.make_gif_from_pil_images(viz, "out_clean_scene.gif")

In [15]:
static = jnp.repeat(images[0,...][jnp.newaxis,...], frames, axis=0)

In [16]:
viz = [b.get_depth_image(1.0 - point_light_image * 1.0, cmap=matplotlib.colormaps['Greys']) for point_light_image in jnp.concatenate((static, images_no_noise, images),axis=2)]
b.make_gif_from_pil_images(viz, "out_merge_scene.gif")

In [17]:
with open("names.csv", "w") as text_file:
    for i, name in enumerate(b.ycb_loader.MODEL_NAMES):
        text_file.write('kinematogram_'+str(i)+', '+name+'\n')